# Pathology training with Mask R-CNN
## Train with malignant and benign class instead of a single mass class

In [1]:
import os
import sys
import itertools
import math
import logging
import json
import re
import random
import time
import concurrent.futures
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import matplotlib.lines as lines
from matplotlib.patches import Polygon
import imgaug
from imgaug import augmenters as iaa

# Root directory of the project
ROOT_DIR = os.path.abspath("../")
# ROOT_DIR = os.getcwd()
print(ROOT_DIR)
# if ROOT_DIR.endswith("samples/nucleus"):
# if ROOT_DIR.endswith("mammography"):
#     # Go up one level to the repo root
#     ROOT_DIR = os.path.dirname(ROOT_DIR)
#     print(ROOT_DIR)
    
# Import Mask RCNN
sys.path.append(ROOT_DIR)
from mrcnn import utils
from mrcnn import visualize
from mrcnn.visualize import display_images
from mrcnn import model as modellib
from mrcnn.model import log

import mammo_baseline_pathology

%matplotlib inline 

/home/liujing/liuyibo/Mammo_MaskRCNN


Using TensorFlow backend.


In [3]:
# Comment out to reload imported modules if they change
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Configurations

In [10]:
# Dataset directory
DATASET_DIR = os.path.join(ROOT_DIR, "dataset/mammo_all")

# Load dataset
subset = "mass_train"
if "3x" in subset:
    augmented=True
else:
    augmented=False

dataset_train = mammo_baseline_pathology.MammoDataset()
dataset_train.load_mammo(DATASET_DIR, subset=subset, augmented=augmented, json_filename="mammo_all_ddsm_mass_train.json")

# Must call before using the dataset
dataset_train.prepare()

print("Image Count: {}".format(len(dataset_train.image_ids)))
print("Class Count: {}".format(dataset_train.num_classes))
for i, info in enumerate(dataset_train.class_info):
    print("{:3}. {:50}".format(i, info['name']))
    

# Load validation dataset
dataset_val = mammo_baseline_pathology.MammoDataset() # May17
# dataset_val = mammo_ddsm.MammoDataset()
dataset_val.load_mammo(DATASET_DIR, "mass_val", augmented=augmented, json_filename="mammo_all_ddsm_mass_train.json")
# dataset_val.load_mammo(DATASET_DIR, "calc_val", augmented=augmented, json_filename="mammo_ddsm_calc_train.json")
dataset_val.prepare()

print("Images: {}\nClasses: {}".format(len(dataset_val.image_ids), dataset_val.class_names))




mass_train
Image Count: 985
Class Count: 3
  0. BG                                                
  1. MALIGNANT                                         
  2. BENIGN                                            
mass_val
Images: 246
Classes: ['BG', 'MALIGNANT', 'BENIGN']


## Notebook Preferences

In [11]:
def get_ax(rows=1, cols=1, size=16):
    """Return a Matplotlib Axes array to be used in
    all visualizations in the notebook. Provide a
    central point to control graph sizes.
    
    Adjust the size attribute to control how big to render images
    """
    _, ax = plt.subplots(rows, cols, figsize=(size*cols, size*rows))
    return ax

In [13]:
#######################################
#   Training with Pathology labelling #
#      images. Max_dim = 1024         # 
#      Training set size = 4x         # 
#######################################



# Configurations
# Use configuation from mammo.py, but override
# image resizing so we see the real sizes here
class NoResizeConfig(mammo_baseline_pathology.MammoConfig):
    BACKBONE = "resnet101"
#     IMAGE_RESIZE_MODE = "none"
#     USE_MINI_MASK = False
    # Adjust depending on your GPU memory
    IMAGES_PER_GPU = 2
    
    IMAGE_MAX_DIM = 1024

    # Number of classes (including background)
    NUM_CLASSES = 1 + 2  # Background + Malignant + Benign

    # Number of training and validation steps per epoch
    STEPS_PER_EPOCH = (len(dataset_train.image_ids)) // IMAGES_PER_GPU
    VALIDATION_STEPS = max(1, len(dataset_val.image_ids) // IMAGES_PER_GPU)
    
    MINI_MASK_SHAPE = (56, 56)  # (height, width) of the mini-mask
    RPN_ANCHOR_SCALES = (32, 64, 128, 256, 512)
    RPN_TRAIN_ANCHORS_PER_IMAGE = 512
    RPN_ANCHOR_RATIOS = [0.5, 1, 2]
    
    # ROIs kept after non-maximum supression (training and inference)
    POST_NMS_ROIS_TRAINING = 2000
    POST_NMS_ROIS_INFERENCE = 1000
    
    # Non-max suppression threshold to filter RPN proposals.
    # You can increase this during training to generate more propsals.
    RPN_NMS_THRESHOLD = 0.7
    
config = NoResizeConfig()
config.display()

MODEL_DIR = 'checkpoints'

# Create model
model = modellib.MaskRCNN(mode="training", config=config,                          
                          model_dir=MODEL_DIR)
# Select weights file to load
weights_path = 'resnet50_weights_tf_dim_ordering_tf_kernels_notop.h5'
# weights_path = model.get_imagenet_weights()
# weights_path = COCO_MODEL_PATH
# model.load_weights(COCO_MODEL_PATH, by_name=True)

# Load weights
print("Loading weights ", weights_path)
model.load_weights(weights_path, by_name=True)

model.train(dataset_train, dataset_val, config.LEARNING_RATE, epochs=10, layers='all')



Configurations:
BACKBONE                       resnet101
BACKBONE_STRIDES               [4, 8, 16, 32, 64]
BATCH_SIZE                     2
BBOX_STD_DEV                   [0.1 0.1 0.2 0.2]
COMPUTE_BACKBONE_SHAPE         None
DETECTION_MAX_INSTANCES        100
DETECTION_MIN_CONFIDENCE       0.7
DETECTION_NMS_THRESHOLD        0.3
FPN_CLASSIF_FC_LAYERS_SIZE     1024
GPU_COUNT                      1
GRADIENT_CLIP_NORM             5.0
IMAGES_PER_GPU                 2
IMAGE_MAX_DIM                  1024
IMAGE_META_SIZE                15
IMAGE_MIN_DIM                  512
IMAGE_MIN_SCALE                0
IMAGE_RESIZE_MODE              square
IMAGE_SHAPE                    [1024 1024    3]
LEARNING_MOMENTUM              0.9
LEARNING_RATE                  0.001
LOSS_WEIGHTS                   {'rpn_class_loss': 1.0, 'rpn_bbox_loss': 1.0, 'mrcnn_class_loss': 1.0, 'mrcnn_bbox_loss': 1.0, 'mrcnn_mask_loss': 1.0}
MASK_POOL_SIZE                 14
MASK_SHAPE                     [28, 28]
MAX_GT_INSTA

/home/liujing/anaconda3/envs/keras/lib/python3.6/site-packages/tensorflow/python/ops/gradients_impl.py:112: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "
/home/liujing/anaconda3/envs/keras/lib/python3.6/site-packages/keras/engine/training.py:2087: UserWarning: Using a generator with `use_multiprocessing=True` and multiple workers may duplicate your data. Please consider using the`keras.utils.Sequence class.
  UserWarning('Using a generator with `use_multiprocessing=True`'


Epoch 1/10
 87/492 [====>.........................] - ETA: 13:18 - loss: 1.7731 - rpn_class_loss: 0.1691 - rpn_bbox_loss: 0.7968 - mrcnn_class_loss: 0.0284 - mrcnn_bbox_loss: 0.4229 - mrcnn_mask_loss: 0.3559

ERROR:root:Error processing image {'id': 'Mass-Training_P_00076_LEFT_CC', 'source': 'pathology', 'path': '/home/liujing/liuyibo/Mammo_MaskRCNN/dataset/mammo_all/mass_train/Mass-Training_P_00076_LEFT_CC/full_image/Mass-Training_P_00076_LEFT_CC.png', 'pathology': ['BENIGN'], 'catID': [2]}
Traceback (most recent call last):
  File "/home/liujing/anaconda3/envs/keras/lib/python3.6/site-packages/PIL/ImageFile.py", line 224, in load
    s = read(self.decodermaxblock)
  File "/home/liujing/anaconda3/envs/keras/lib/python3.6/site-packages/PIL/PngImagePlugin.py", line 652, in load_read
    cid, pos, length = self.png.read()
  File "/home/liujing/anaconda3/envs/keras/lib/python3.6/site-packages/PIL/PngImagePlugin.py", line 122, in read
    length = i32(s)
  File "/home/liujing/anaconda3/envs/keras/lib/python3.6/site-packages/PIL/_binary.py", line 77, in i32be
    return unpack_from(">I", c, o)[0]
struct.error: unpack_from requires a buffer of at least 4 bytes

During handling of the above except

303/492 [=================>............] - ETA: 4:23 - loss: 1.7185 - rpn_class_loss: 0.0730 - rpn_bbox_loss: 0.5920 - mrcnn_class_loss: 0.0261 - mrcnn_bbox_loss: 0.5051 - mrcnn_mask_loss: 0.5223

...

485/492 [============================>.] - ETA: 8s - loss: 1.0899 - rpn_class_loss: 0.0088 - rpn_bbox_loss: 0.2221 - mrcnn_class_loss: 0.0735 - mrcnn_bbox_loss: 0.3033 - mrcnn_mask_loss: 0.4822

ERROR:root:Error processing image {'id': 'Mass-Training_P_00076_LEFT_CC', 'source': 'pathology', 'path': '/home/liujing/liuyibo/Mammo_MaskRCNN/dataset/mammo_all/mass_train/Mass-Training_P_00076_LEFT_CC/full_image/Mass-Training_P_00076_LEFT_CC.png', 'pathology': ['BENIGN'], 'catID': [2]}
Traceback (most recent call last):
  File "/home/liujing/anaconda3/envs/keras/lib/python3.6/site-packages/PIL/ImageFile.py", line 224, in load
    s = read(self.decodermaxblock)
  File "/home/liujing/anaconda3/envs/keras/lib/python3.6/site-packages/PIL/PngImagePlugin.py", line 652, in load_read
    cid, pos, length = self.png.read()
  File "/home/liujing/anaconda3/envs/keras/lib/python3.6/site-packages/PIL/PngImagePlugin.py", line 122, in read
    length = i32(s)
  File "/home/liujing/anaconda3/envs/keras/lib/python3.6/site-packages/PIL/_binary.py", line 77, in i32be
    return unpack_from(">I", c, o)[0]
struct.error: unpack_from requires a buffer of at least 4 bytes

During handling of the above except

492/492 [==============================] - 684s 1s/step - loss: 1.0915 - rpn_class_loss: 0.0088 - rpn_bbox_loss: 0.2215 - mrcnn_class_loss: 0.0738 - mrcnn_bbox_loss: 0.3037 - mrcnn_mask_loss: 0.4836 - val_loss: 1.2176 - val_rpn_class_loss: 0.0085 - val_rpn_bbox_loss: 0.3247 - val_mrcnn_class_loss: 0.0788 - val_mrcnn_bbox_loss: 0.3410 - val_mrcnn_mask_loss: 0.4647
Epoch 7/10
492/492 [==============================] - 662s 1s/step - loss: 1.0714 - rpn_class_loss: 0.0076 - rpn_bbox_loss: 0.2138 - mrcnn_class_loss: 0.0805 - mrcnn_bbox_loss: 0.2980 - mrcnn_mask_loss: 0.4714 - val_loss: 1.1800 - val_rpn_class_loss: 0.0075 - val_rpn_bbox_loss: 0.2993 - val_mrcnn_class_loss: 0.0823 - val_mrcnn_bbox_loss: 0.3256 - val_mrcnn_mask_loss: 0.4653
Epoch 8/10
446/492 [==========================>...] - ETA: 53s - loss: 0.9973 - rpn_class_loss: 0.0073 - rpn_bbox_loss: 0.1814 - mrcnn_class_loss: 0.0791 - mrcnn_bbox_loss: 0.2782 - mrcnn_mask_loss: 0.4514

ERROR:root:Error processing image {'id': 'Mass-Training_P_00076_LEFT_CC', 'source': 'pathology', 'path': '/home/liujing/liuyibo/Mammo_MaskRCNN/dataset/mammo_all/mass_train/Mass-Training_P_00076_LEFT_CC/full_image/Mass-Training_P_00076_LEFT_CC.png', 'pathology': ['BENIGN'], 'catID': [2]}
Traceback (most recent call last):
  File "/home/liujing/anaconda3/envs/keras/lib/python3.6/site-packages/PIL/ImageFile.py", line 224, in load
    s = read(self.decodermaxblock)
  File "/home/liujing/anaconda3/envs/keras/lib/python3.6/site-packages/PIL/PngImagePlugin.py", line 652, in load_read
    cid, pos, length = self.png.read()
  File "/home/liujing/anaconda3/envs/keras/lib/python3.6/site-packages/PIL/PngImagePlugin.py", line 122, in read
    length = i32(s)
  File "/home/liujing/anaconda3/envs/keras/lib/python3.6/site-packages/PIL/_binary.py", line 77, in i32be
    return unpack_from(">I", c, o)[0]
struct.error: unpack_from requires a buffer of at least 4 bytes

During handling of the above except

492/492 [==============================] - 661s 1s/step - loss: 0.9910 - rpn_class_loss: 0.0072 - rpn_bbox_loss: 0.1788 - mrcnn_class_loss: 0.0791 - mrcnn_bbox_loss: 0.2764 - mrcnn_mask_loss: 0.4495 - val_loss: 1.1649 - val_rpn_class_loss: 0.0069 - val_rpn_bbox_loss: 0.2985 - val_mrcnn_class_loss: 0.0843 - val_mrcnn_bbox_loss: 0.3225 - val_mrcnn_mask_loss: 0.4527
Epoch 9/10
492/492 [==============================] - 666s 1s/step - loss: 0.9610 - rpn_class_loss: 0.0071 - rpn_bbox_loss: 0.1631 - mrcnn_class_loss: 0.0821 - mrcnn_bbox_loss: 0.2609 - mrcnn_mask_loss: 0.4478 - val_loss: 1.2241 - val_rpn_class_loss: 0.0068 - val_rpn_bbox_loss: 0.3506 - val_mrcnn_class_loss: 0.0798 - val_mrcnn_bbox_loss: 0.3325 - val_mrcnn_mask_loss: 0.4545
Epoch 10/10
492/492 [==============================] - 670s 1s/step - loss: 0.9110 - rpn_class_loss: 0.0066 - rpn_bbox_loss: 0.1554 - mrcnn_class_loss: 0.0717 - mrcnn_bbox_loss: 0.2457 - mrcnn_mask_loss: 0.4317 - val_loss: 1.2893 - val_rpn_class_loss: 0.007